# Modeling- Part 2 (Classification)

In [1]:
import os

import pandas as pd
import numpy as np

import pickle
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve, cross_val_score, RandomizedSearchCV, validation_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from bayes_opt import BayesianOptimization
import lightgbm as lgb

from sklearn.feature_selection import mutual_info_regression, SelectKBest, f_regression
from sklearn.utils import resample, shuffle
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder


from sklearn.metrics import confusion_matrix
random_state=42

In [2]:
# To suppress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [3]:
df= pd.read_csv('/Users/josephlim/Desktop/Data Science/Capstone Projects/Capstone project- Spotify/spotify_data_preprocessed_exp.csv')

In [4]:
df.shape

(587927, 12)

In [5]:
df.head()

,popularity,duration_ms,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,0.06,126903,0.645,0.4450,-13.338,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,0.00,98200,0.695,0.2630,-22.136,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,0.00,181640,0.434,0.1770,-21.180,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,0.00,176907,0.321,0.0946,-27.961,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,0.00,163080,0.402,0.1580,-16.900,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587927 entries, 0 to 587926
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   popularity        587927 non-null  float64
 1   duration_ms       587927 non-null  int64  
 2   danceability      587927 non-null  float64
 3   energy            587927 non-null  float64
 4   loudness          587927 non-null  float64
 5   speechiness       587927 non-null  float64
 6   acousticness      587927 non-null  float64
 7   instrumentalness  587927 non-null  float64
 8   liveness          587927 non-null  float64
 9   valence           587927 non-null  float64
 10  tempo             587927 non-null  float64
 11  time_signature    587927 non-null  int64  
dtypes: float64(10), int64(2)
memory usage: 53.8 MB


## Classifications
We'll try to predict songs' popularities by categorizing popularities(into "high","mid","low"), and classifying songs into those categories.

In [7]:
pd.cut(df['popularity'], bins=3)

0         (-0.001, 0.333]
1         (-0.001, 0.333]
2         (-0.001, 0.333]
3         (-0.001, 0.333]
4         (-0.001, 0.333]
               ...       
587922    (-0.001, 0.333]
587923    (-0.001, 0.333]
587924    (-0.001, 0.333]
587925    (-0.001, 0.333]
587926    (-0.001, 0.333]
Name: popularity, Length: 587927, dtype: category
Categories (3, interval[float64, right]): [(-0.001, 0.333] < (0.333, 0.667] < (0.667, 1.0]]

In [8]:
labels=['low','medium', 'high']
df['popularity']= pd.cut(df['popularity'], bins=3, labels=labels, right=True)

In [9]:
df.popularity.value_counts()

low       363052
medium    213090
high       11785
Name: popularity, dtype: int64

The dataset is unbalanced. This is intuitive, because there aren't as many popular songs as there are non-popular songs (otherwise, there will be much more financial stability in music industry!). However,imbalance in dataset will tamper with the accuracy of our model. One way to counteract this is by upsampling songs with high popularity.  We will then perform K-Nearest Neighbor classification, because they are good at handling noisy data. 

### Up-sampling songs

In [10]:
df_high= df[df.popularity=='high']
df_mid= df[df.popularity=='medium']
df_low= df[df.popularity=='low']

In [11]:
# df_low_upsampled= resample(df_low, replace=True, n_samples= 362633, random_state=42)
df_mid_upsampled= resample(df_mid, replace=True, n_samples= 363052, random_state=42)
df_high_upsampled= resample(df_high, replace=True, n_samples= 363052, random_state=42)

# list_df_upsampled_tomid=[df_high_upsampled, df_mid, df_low_upsampled]
list_df_upsampled_tohigh=[df_high_upsampled, df_mid_upsampled, df_low]

# df_resampled= pd.concat(list_df_upsampled_tomid)
df_resampled= pd.concat(list_df_upsampled_tohigh)

In [12]:
df_resampled.popularity.value_counts()

low       363052
medium    363052
high      363052
Name: popularity, dtype: int64

In [13]:
X_re= df_resampled.drop('popularity', axis=1)
y_re= df_resampled['popularity']

In [14]:
X_train, X_test, y_train, y_test= train_test_split(X_re, y_re, random_state= 42, test_size=0.3)

### Baseline Model- Random Prediction 
<br> We will randomly pull samples to serve as a baseline model to compare against different models we train. 

In [15]:
y_test_rand= shuffle(y_test)

In [16]:
print(classification_report(y_test, y_test_rand))

              precision    recall  f1-score   support

        high       0.33      0.33      0.33    108764
         low       0.33      0.33      0.33    108706
      medium       0.33      0.33      0.33    109277

    accuracy                           0.33    326747
   macro avg       0.33      0.33      0.33    326747
weighted avg       0.33      0.33      0.33    326747



### Testing Different Classification Models

### K-Nearest Neighbors (KNN) Classifier

In [17]:
KNN= KNeighborsClassifier()

KNN.fit(X_train, y_train)

y_pred_classification= KNN.predict(X_test)
print(classification_report(y_test, y_pred_classification))

              precision    recall  f1-score   support

        high       0.88      1.00      0.94    108764
         low       0.66      0.54      0.60    108706
      medium       0.63      0.65      0.64    109277

    accuracy                           0.73    326747
   macro avg       0.72      0.73      0.72    326747
weighted avg       0.72      0.73      0.72    326747



Because KNN is not a tree-based algorithm, it requires standardization. We will put it as part of our function.

In [18]:
def fit_model(n_neighbors):
    StandardScaler()
    n_neighbors= round(n_neighbors)

    regressor= KNeighborsClassifier(n_neighbors= n_neighbors)

    return np.mean(cross_validate(regressor, X_train, y_train, scoring='accuracy', error_score= 'raise', cv=5)['test_score'])

In [19]:
KNN_BO = BayesianOptimization(fit_model,{
        'n_neighbors': (1,80)
    })

KNN_BO.maximize(n_iter=10, init_points=2, allow_duplicate_points= True)

|   iter    |  target   | n_neig... |
-------------------------------------


/var/folders/z9/t15ddpld70nc5qf41442lpr80000gn/T/ipykernel_6252/3189612465.py:5: DeprecationWarning: 
Passing acquisition function parameters or gaussian process parameters to maximize
is no longer supported, and will cause an error in future releases. Instead,
please use the "set_gp_params" method to set the gp params, and pass an instance
 of bayes_opt.util.UtilityFunction using the acquisition_function argument

  KNN_BO.maximize(n_iter=10, init_points=2, allow_duplicate_points= True)


| 1         | 0.6682    | 10.72     |
| 2         | 0.5155    | 55.57     |
| 3         | 0.6612    | 12.03     |
| 4         | 0.8009    | 1.0       |
| 5         | 0.4981    | 80.0      |
| 6         | 0.5641    | 32.02     |
| 7         | 0.7497    | 2.228     |
| 8         | 0.6137    | 21.23     |
| 9         | 0.5303    | 43.2      |
| 10        | 0.5077    | 67.55     |
| 11        | 0.8009    | 1.011     |
| 12        | 0.5896    | 26.26     |


In [20]:
print(KNN_BO.max)

{'target': 0.8008759069757829, 'params': {'n_neighbors': 1.0}}


In [21]:
n_neighbors= KNN_BO.max['params']['n_neighbors']

In [22]:
model = KNeighborsClassifier(n_neighbors= round(n_neighbors))
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

        high       0.99      0.96      0.98    112029
         low       0.66      0.80      0.72     90009
      medium       0.82      0.72      0.76    124709

    accuracy                           0.82    326747
   macro avg       0.82      0.82      0.82    326747
weighted avg       0.83      0.82      0.82    326747



It's odd that our test performed better than our hyperparameter tuning. We will try n_neighbor with the next best accuracy: 2.228.

In [23]:
model = KNeighborsClassifier(n_neighbors= round(2.228))
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

        high       1.00      0.93      0.96    117017
         low       0.74      0.66      0.70    123205
      medium       0.58      0.73      0.64     86525

    accuracy                           0.77    326747
   macro avg       0.77      0.77      0.77    326747
weighted avg       0.79      0.77      0.78    326747



WOW! This is a huge improvement (though it probably is overfitting). Let's try Random Forest Classification.

### Random Forest Classifier

In [24]:
RFC= RandomForestClassifier(random_state= random_state)

RFC.fit(X_train, y_train)
y_rfc_pred= RFC.predict(X_test)
print(classification_report(y_rfc_pred, y_test))

              precision    recall  f1-score   support

        high       1.00      0.99      0.99    110109
         low       0.80      0.86      0.83    100943
      medium       0.86      0.81      0.84    115695

    accuracy                           0.89    326747
   macro avg       0.89      0.89      0.89    326747
weighted avg       0.89      0.89      0.89    326747



Clearly, Random Forest Classifier performs best for our case. Let's use this model. Before fully delving into this model though, let's check for feature importance to see which features are more relevant for our task.

#### Random Forest Classification- Bayesian Optimization

In [25]:
def fit_model(n_estimators, min_samples_split, max_depth, max_leaf_nodes):
    n_estimators= round(n_estimators)
    min_samples_split= round(min_samples_split) 
    max_depth= round(max_depth)
    max_leaf_nodes= round(max_leaf_nodes)

    regressor= RandomForestClassifier(n_estimators= n_estimators, 
                                 min_samples_split= min_samples_split,
                                 max_depth=max_depth,
                                 random_state=random_state)

    return np.mean(cross_validate(regressor, X_train, y_train, scoring='accuracy', error_score= 'raise', cv=5)['test_score'])

In [26]:
rf_BO = BayesianOptimization(fit_model,{
        'n_estimators': (1,1000),
        'min_samples_split':(1.5,100), 
        'max_depth': (1,10),
        'max_leaf_nodes': (2,10)
    })

rf_BO.maximize(n_iter=10, init_points=2)

|   iter    |  target   | max_depth | max_le... | min_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | 0.5957    | 8.544     | 9.308     | 95.7      | 354.7     |
| 2         | 0.5472    | 5.192     | 2.525     | 56.53     | 796.4     |
| 3         | 0.5832    | 8.414     | 7.696     | 92.3      | 352.5     |
| 4         | 0.559     | 5.868     | 4.078     | 93.66     | 362.2     |
| 5         | 0.5708    | 6.733     | 8.303     | 96.72     | 349.1     |
| 6         | 0.5832    | 8.161     | 7.682     | 94.6      | 355.9     |
| 7         | 0.5078    | 1.549     | 8.412     | 91.52     | 352.7     |
| 8         | 0.5831    | 7.834     | 2.33      | 98.04     | 352.5     |
| 9         | 0.6116    | 9.661     | 2.18      | 92.31     | 350.9     |
| 10        | 0.5957    | 9.294     | 3.454     | 95.53     | 343.0     |
| 11        | 0.5958    | 8.623     | 2.321     | 89.05     | 344.4     |
| 12        | 0.6116    | 9.583     | 

In [27]:
print(rf_BO.max)

{'target': 0.6116375865886491, 'params': {'max_depth': 9.660960549099896, 'max_leaf_nodes': 2.180096037376238, 'min_samples_split': 92.30703719171741, 'n_estimators': 350.93374603471426}}


Let's set optimized parameteres into variables.

In [28]:
max_depth= rf_BO.max['params']['max_depth']
max_leaf_nodes= rf_BO.max['params']['max_leaf_nodes']
min_samples_split= rf_BO.max['params']['min_samples_split']
n_estimators= rf_BO.max['params']['n_estimators']

In [29]:
model = RandomForestClassifier(max_depth= round(max_depth),
    max_leaf_nodes= round(max_leaf_nodes),
    min_samples_split= round(min_samples_split),
    n_estimators=round(n_estimators))
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

        high       0.79      0.46      0.58    187189
         low       0.68      0.53      0.60    139550
      medium       0.00      0.62      0.00         8

    accuracy                           0.49    326747
   macro avg       0.49      0.54      0.39    326747
weighted avg       0.74      0.49      0.59    326747



Through hyperparameter tuning, we learned that random forest classification model can expect target score of 0.61. This score is reasonable, as the model is expected to perform better on training set. However, our current target score is still great. 

Clearly, KNN classifier performs better, with accuracy score of 0.82. Although we have a model performance of 0.82, it is possible that the model is overfitting. However, given different iterations of the model, you can expect it to reliably predict anywhere between 65 to 75% accuracy. We have come a long way from the baseline model of 0.33 accuracy.